In [1]:
import requests
import json
import numpy as np

from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score

import math

from sklearn.metrics import f1_score

In [2]:
model = SentenceTransformer('all-MiniLM-L6-v2')

characters = ['Monica Geller', 'Joey Tribbiani', 'Chandler Bing', 'Phoebe Buffay', 'Ross Geller', 'Rachel Green', 'others']

#characters = ['Monica Geller', 'Joey Tribbiani', 'Chandler Bing', 'Phoebe Buffay', 'Ross Geller', 'Rachel Green']

sentences = []

for character in characters:
    f = open("embeddings1/" + character +"100"+ ".txt", "r")
    lines = f.readlines()
    sentences.append(lines)

i=0
embeddings = {}
for character in characters:
    embeddings[character] = model.encode(sentences[i])
    i+=1


#to confirm number of lines used to create the embedding
print(len(embeddings['Monica Geller']))

for character in characters:

    mean = embeddings[character][0]
    for embedding in embeddings[character]:
        mean += embedding
    
    embeddings[character] = mean



100


In [6]:
#test set 1 = season 8

with open('sets/test_set1.json') as f:
    test_set1 = json.load(f)

with open('sets/train_set1.json') as f:
    train_set1 = json.load(f)

################################################################

#test set 2 = 10% of each season

with open('sets/test_set2.json') as f:
    test_set2 = json.load(f)

with open('sets/train_set2.json') as f:
    train_set2 = json.load(f)


#CHANGE HERE
test_set = test_set2
train_set = train_set2

In [2]:
def tf_idf_calculator(lines):    
    # Compute the term frequency of each word in each sentence
    tf = []
    
    for sentence in lines:
        tf_sentence = {}
        for word in sentence.split():
            tf_sentence[word] = tf_sentence.get(word, 0) + 1
        tf.append(tf_sentence)

    # Compute the inverse document frequency of each word
    idf = {}
    for sentence in lines:
        for word in sentence.split():
            idf[word] = idf.get(word, 0) + 1
    for word in idf:
        idf[word] = math.log(len(lines) / idf[word])

    # Compute the tf-idf of each sentence
    tf_idf = []
    for i, sentence in enumerate(lines):
        tf_idf_sentence = 0
        for word in tf[i]:
            tf_idf_sentence += tf[i][word] * idf[word]
        tf_idf.append(tf_idf_sentence)
    
    return tf_idf

In [ ]:
all_lines = []
speakers = []


for utterance in test_set:
    if utterance['speakers'] != []:
        all_lines.append(utterance['transcript'])

for utterance in test_set:
    if utterance['speakers'] != []:
        speakers.append(utterance['speakers'][0])

#remove duplicates
#all_lines = list( dict.fromkeys(all_lines) )


tf_idf = tf_idf_calculator(all_lines)

all_lines_tf_idf = []

#format: all_lines_tf_idf = [...,[tf-idf score ,sentence],...]
for i,sentence in enumerate(all_lines):
    cell = []
    cell= [tf_idf[i], sentence, speakers[i]]
    all_lines_tf_idf.append(cell)
    
# lower -> higher     
sorted_list = sorted(all_lines_tf_idf)

tf_idfs_utterances = []

#get only the strings
for elem in sorted_list:
    tf_idfs_utterances.append(elem[1])


for elem in sorted_list:
    print(elem)


In [7]:
def compute_similarity(vector):

    angles = {}

    for character in characters:
        angles[character] = cosine_similarity(vector.reshape(1,-1), embeddings[character].reshape(1,-1))[0][0]

    
    
    #Smaller angles between vectors produce larger cosine values, indicating greater cosine similarity

    character = [i for i in angles if angles[i]==max(angles.values())]


    return character[0]

In [104]:
predicted = []
real = []

correct = 0
total = 0

for utterance in test_set:
    if len(utterance['speakers']) == 1:

        total += 1

        line_embed = model.encode(utterance['transcript'])
    
        pred = compute_similarity(line_embed)

        predicted.append(pred)
        real.append(utterance['speakers'][0])

        if pred == utterance['speakers'][0]:
            correct+=1
        elif pred == "others" and utterance['speakers'][0] not in ['Monica Geller', 'Joey Tribbiani', 'Chandler Bing', 'Phoebe Buffay', 'Ross Geller', 'Rachel Green']:
            correct+=1

                    
print(accuracy_score(real, predicted)*100)

accuracy = (float(correct)/total)*100

print("accuracy: ", accuracy)

print("f1 score: ", f1_score(real, predicted, average='macro'))

print(len(characters))

15.067147068457254
accuracy:  19.84932852931543
f1 score:  0.0034750003354247954
7


classifier WITH filter in the test set

In [15]:
predicted = []
real = []

correct = 0
total = 0


#threshold = 80
threshold = 30

all = 0
for utterance in test_set:
    if len(utterance['speakers']) == 1:
        all += 1

        index = tf_idfs_utterances.index(utterance['transcript'])

        #if sorted_list[index][0] <= threshold:
        if sorted_list[index][0] >= threshold:

            total += 1

            line_embed = model.encode(utterance['transcript'])
        
            pred = compute_similarity(line_embed)

            predicted.append(pred)
            real.append(utterance['speakers'][0])

            if pred == utterance['speakers'][0]:
                correct+=1
            elif pred == "others" and utterance['speakers'][0] not in ['Monica Geller', 'Joey Tribbiani', 'Chandler Bing', 'Phoebe Buffay', 'Ross Geller', 'Rachel Green']:
                correct+=1


                    
print(accuracy_score(real, predicted)*100)

accuracy = (float(correct)/total)*100

print(accuracy)
print(len(characters))
print(threshold)

print(100 - float(total/all)*100)

14.319952067106051
18.184541641701617
7
30
46.25664144260183
